https://huggingface.co/docs/transformers/en/training

https://huggingface.co/datasets/Yelp/yelp_review_full

https://huggingface.co/google-bert/bert-base-cased

In [1]:
from datasets import load_dataset

## dataset

In [2]:
dataset = load_dataset("yelp_review_full")

In [3]:
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [4]:
from transformers import AutoTokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased", clean_up_tokenization_spaces=True)

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [7]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [8]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))

In [9]:
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

## train

In [10]:
from transformers import AutoModelForSequenceClassification

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5, device_map="cuda")

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [12]:
def evaluate(s: str):
    inputs = tokenizer(s, return_tensors="pt")
    inputs = inputs.to("cuda")
    model_outputs = model(**inputs)
    return model_outputs[0]

In [13]:
evaluate("that fish is super tasty, I liked it so much. Highly recommend")

tensor([[-0.5368,  0.0784, -0.9938,  0.1739, -0.4372]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [14]:
evaluate("the worst fish I've ever eaten, can't imaging how they cooked")

tensor([[-0.4129,  0.0464, -1.0037,  0.1865, -0.4272]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [15]:
evaluate("not terrible, I eated it but it could be better")

tensor([[-0.4684,  0.0070, -0.9820,  0.1507, -0.4008]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

## train

In [16]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [17]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer", 
    eval_strategy="epoch",
)

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.078117,0.546000
2,No log,1.020385,0.565000
3,No log,1.056581,0.589000


TrainOutput(global_step=375, training_loss=0.9636922200520833, metrics={'train_runtime': 328.523, 'train_samples_per_second': 9.132, 'train_steps_per_second': 1.141, 'total_flos': 789354427392000.0, 'train_loss': 0.9636922200520833, 'epoch': 3.0})

## check

In [23]:
def evaluate2(s: str):
    inputs = tokenizer(s, return_tensors="pt")
    inputs = inputs.to("cuda")
    model_outputs = model(**inputs)
    return model_outputs[0]

5 logits are 1-start up to 5-star

In [24]:
evaluate2("that fish is super tasty, I liked it so much. Highly recommend")

tensor([[-1.9205, -2.4586, -0.7762,  2.4402,  2.9567]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [25]:
evaluate2("the worst fish I've ever eaten, can't imaging how they cooked")

tensor([[ 3.3060,  1.5929, -0.6564, -1.9586, -1.5000]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [26]:
evaluate2("not terrible, I eated it but it could be better")

tensor([[-1.6759,  1.6243,  2.6040, -0.1788, -2.1950]], device='cuda:0',
       grad_fn=<AddmmBackward0>)